In [53]:
import numpy as np
from rank_bm25 import BM25Okapi
from fastbm25 import fastbm25
from retriv import SparseRetriever


class BM25_sparse:
   def __init__(self):
      self.docs = []
      self.query = []
      self.news = []
      self.result = []


   def setQuery(self, query):
      self.query = query


   def setDocs(self, noticias):
      self.news = list(set(noticias))  # removiendo duplicados
      self.docs = [n.getCuerpo() for n in self.news]


   def apply(self):
      tmp = self.news
      sr = SparseRetriever(
        index_name="new-index",
        model="bm25",
        min_df=1,
        tokenizer="whitespace",
        stemmer="spanish",
        stopwords="spanish",
        do_lowercasing=True,
        do_ampersand_normalization=True,
        do_special_chars_normalization=True,
        do_acronyms_normalization=True,
        do_punctuation_removal=True
      )
      docs = []
      for i in range(len(self.docs)):
         docs.append({'id': 'doc_' + str(i + 1), 'text': self.docs[i]})

      # print('DOCS SIZE =', len(docs))

      sr.index(docs)
      bm25_tmp = sr.search(query=self.query, return_docs=True, cutoff=100)

      # print('bm25_tmp (SIZE: ',len(bm25_tmp), ') = ', bm25_tmp)

      doc_scores = [n['score'] for n in bm25_tmp]

      # print('doc_scores (SIZE: ',len(doc_scores), ') = ', doc_scores)

      noticias_relevantes = []
      for i in range(len(doc_scores)):
         if doc_scores[i] > 0:
            tmp[i].setRelevancia(doc_scores[i])
            noticias_relevantes.append(tmp[i])
      self.result = sorted(set(noticias_relevantes), key=lambda x: x.getRelevancia(), reverse = True)

      for n in self.result:
         print('{\"score\":', n.getRelevancia(), ',\n\"título\":\"', n.getTitulo(), '\",\n\"url\":\"', n.getURL(), '\"},')

      return self.result










class BM25_fast:
   def __init__(self):
      self.tokenized_corpus = []
      self.docs = []
      self.query = []
      self.news = []
      self.result = []


   def setQuery(self, query):
      self.query = query.split(" ")


   def setDocs(self, noticias):
      # self.docs = []
      # self.tokenized_corpus = []

      noticias = set(noticias) # removiendo duplicados
      for n in noticias:
         self.tokenized_corpus.append(n.getCuerpo().split(" "))
         self.docs.append(n.getCuerpo())
      self.news = noticias


   def apply(self):
      tmp = list(self.news)
      noticias_relevantes = []
      bm25 = fastbm25(self.tokenized_corpus)
      bm25_tmp = bm25.top_k_sentence(self.query, k=len(self.tokenized_corpus))
      doc_scores = [n[2] for n in bm25_tmp]

      # print('doc_scores (SIZE: ',len(doc_scores), ') = ', doc_scores)

      for i in range(len(doc_scores)):
         if doc_scores[i] > 0:
            tmp[i].setRelevancia(doc_scores[i])
            noticias_relevantes.append(tmp[i])
      self.result = sorted(set(noticias_relevantes), key=lambda x: x.getRelevancia(), reverse = True)
      
      for n in self.result:
         print('{\"score\":', n.getRelevancia(), ',\n\"título\":\"', n.getTitulo(), '\",\n\"url\":\"', n.getURL(), '\"},')

      return self.result












class BM25_okapi:
   def __init__(self):
      self.tokenized_corpus = []
      self.N = 0
      self.avgdl = 0
      self.docs = []
      self.query = []
      self.news = []
      self.result = []


   def setQuery(self, query):
      self.query = query.split(" ")


   def setDocs(self, noticias):
      self.docs = []
      self.tokenized_corpus = []
      noticias = set(noticias) # removiendo duplicados
      for n in noticias:
         self.tokenized_corpus.append(n.getCuerpo().split(" "))
         self.docs.append(n.getCuerpo())
      self.news = noticias


   def apply(self):
      tmp = list(self.news)
      noticias_relevantes = []
      self.N = len(self.docs)
      self.avgdl = sum(len(doc) for doc in self.docs) / self.N
      bm25 = BM25Okapi(self.tokenized_corpus)
      doc_scores = bm25.get_scores(self.query)

      #print('doc_scores (SIZE: ',len(doc_scores), ') = ', doc_scores)

      for i in range(len(doc_scores)):
         if doc_scores[i] > 0:
            tmp[i].setRelevancia(doc_scores[i])
            noticias_relevantes.append(tmp[i])
      self.result = sorted(set(noticias_relevantes), key=lambda x: x.getRelevancia(), reverse = True)

      for n in self.result:
         print('{\"score\":', n.getRelevancia(), ',\n\"título\":\"', n.getTitulo(), '\",\n\"url\":\"', n.getURL(), '\"},')

      return self.result












class BM25_manual:
   def __init__(self):
      self.N = 0
      self.avgdl = 0
      self.docs = []
      self.query = []
      self.news = []
      self.result = []


   def setDocs(self, noticias):
      noticias = set(noticias) # removiendo duplicados
      for n in noticias:
         self.docs.append(n.getCuerpo())
      self.news = noticias


   def setQuery(self, query):
      self.query = query


   def init(self):
      self.N = len(self.docs)
      self.avgdl = sum(len(sentence) for sentence in self.docs) / self.N


   def calculate(self, word, sentence, k=1, b=0.75):
      #term frequency
      freq = sentence.count(word)  # or f(q,D) - freq of query in Doc
      tf = (freq * (k + 1)) / (freq + k * (1 - b + b * len(sentence) / self.avgdl))

      # inverse document frequency...
      N_q = sum([1 for doc in self.docs if word in doc])  # number of docs that contain the word
      idf = np.log(((self.N - N_q + 0.5) / (N_q + 0.5)) + 1)
      return round(tf*idf, 4)


   def getResult(self):
      return self.result


   def apply(self):
      self.init()
      noticias_relevantes = []
      
      # CÓDIGO MODIFICADO
      for doc in self.news:
         relevancia = 0.0
         for word in self.query:
            relevancia = relevancia + self.calculate( word, doc.getCuerpo() )
         if relevancia > 0:
            doc.setRelevancia(relevancia)
            noticias_relevantes.append(doc)

      self.result = sorted(set(noticias_relevantes), key=lambda x: x.getRelevancia(), reverse = True)

      for n in self.result:
         print('{\"score\":', n.getRelevancia(), ',\n\"título\":\"', n.getTitulo(), '\",\n\"url\":\"', n.getURL(), '\"},')

      return self.result

In [5]:
# Clase para almacenar la información de la noticia
class Noticia:
   def __init__(self, titulo, cuerpo):
      self.titulo = titulo
      self.cuerpo = cuerpo
      self.relevancia = 0
      self.url = ''

   def __eq__(self, other):
      return self.titulo==other.titulo

   def __hash__(self):
      return hash(('titulo', self.titulo))

   def setRelevancia(self, relevancia):
      self.relevancia = relevancia

   def getTitulo(self):
      return self.titulo

   def getCuerpo(self):
      return self.cuerpo

   def setURL(self, url):
      self.url = url

   def setCuerpo(self, cuerpo):
      if type(cuerpo) is list:
         cuerpo = " ".join(cuerpo)
      self.cuerpo = cuerpo

   def getRelevancia(self):
      return self.relevancia

   def getURL(self):
      return self.url

In [11]:
from bs4 import BeautifulSoup# Para hacer html parsing
from inscriptis import get_text
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
dirname, scriptname = os.path.split(os.path.abspath('C:/Users/L30cg/OneDrive/Documentos/Residencia/Proyecto/curaciones/core/functions/google_api_bm25.py'))
THIS_DIRECTORY = f'{dirname}{os.sep}'
HEADLESS = False
DRIVER = None


def load_page(url):
   '''Load the specified page with an automated browser'''
   global DRIVER
   if DRIVER is None:
      service = Service(executable_path=f'{THIS_DIRECTORY}chromedriver.exe')
      options = webdriver.ChromeOptions()
      options.headless = HEADLESS
      options.add_experimental_option('excludeSwitches', ['enable-automation'])
      options.add_experimental_option('excludeSwitches', ['enable-logging'])
      options.add_experimental_option('useAutomationExtension', False)
      DRIVER = webdriver.Chrome(options=options, service=service)
   DRIVER.get(url)


def obtener_noticias_completas(urls):
    resultados = []
    for url in urls:
        load_page(url)
        html    = DRIVER.page_source
        soup    = BeautifulSoup(html, "html.parser")# Para obtener el titulo de la noticia
        texto   = get_text(html)
        titulo  = '' if soup.title is None else soup.title.string
        noticia = Noticia(titulo, texto)
        noticia.setURL(url)
        resultados.append(noticia)
    return resultados

In [17]:
urls = [
    [
        'https://factual.afp.com/beber-agua-caliente-sola-o-con-limon-no-cura-el-cancer',
        'https://factual.afp.com/consumir-mucha-vitamina-c-no-impide-contagiarse-del-nuevo-coronavirus',
        'https://www.facebook.com/AFPespanol/posts/d41d8cd9/2517545981862495/',
        'https://medlineplus.gov/spanish/druginfo/meds/a619019-es.html',
        'https://medlineplus.gov/spanish/druginfo/natural/843.html',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/reflujo-de-acido-gastrico-acidez-estomacal-y-enfermedad-por-reflujo-gastroesofagico-erge-cual-es-la-diferencia',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/cinco-sugerencias-para-las-personas-con-alopecia-areata'
    ],
    [
        'https://factual.afp.com/las-mascarillas-limpias-y-bien-manipuladas-no-provocan-neumonia',
        'https://factual.afp.com/verificaciones-de-la-afp-en-espanol-sobre-el-nuevo-coronavirus',
        'https://chequeado.com/verificacionfb/es-falso-que-el-tapabocas-cultive-hongos-y-bacterias-que-bloquean-el-oxigeno-y-provocan-neumonia/',
        'https://medlineplus.gov/spanish/ency/article/000145.htm',
        'https://medlineplus.gov/spanish/ency/patientinstructions/000452.htm',
        'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102523/',
        'https://www.facebook.com/minsaperu/photos/a.306009007764/10157401567772765/?type=3',
        'https://www.cdc.gov/spanish/index.html'
    ],
    [
        'https://conecta.tec.mx/es/noticias/nacional/salud/mitos-y-preguntas-usuales-del-coronavirus-que-causa-el-covid-19',
        'https://factual.afp.com/la-mezcla-que-incluye-limon-aspirinas-jengibre-cebolla-y-canela-no-cura-el-coronavirus',
        'https://factual.afp.com/no-hay-evidencia-cientifica-de-que-la-cebolla-permita-prevenir-o-curar-el-covid-19',
        'https://www.nidcd.nih.gov/es/espanol/trastornos-del-olfato',
        'https://www.niddk.nih.gov/health-information/informacion-de-la-salud/enfermedades-rinones/ninos/nutricion-ninos-enfermedad-renal-cronica',
        'https://medlineplus.gov/spanish/ency/patientinstructions/000976.htm',
        'https://medlineplus.gov/spanish/druginfo/natural/700.html'
    ],
    [
        'https://factual.afp.com/el-limon-la-aspirina-y-la-miel-no-tienen-efectos-sobre-los-virus',
        'https://factual.afp.com/la-mezcla-que-incluye-limon-aspirinas-jengibre-cebolla-y-canela-no-cura-el-coronavirus',
        'https://www.france24.com/es/20200620-cadena-falsa-aspirina-limon-miel-cura-covid19',
        'https://www.health.ny.gov/es/diseases/communicable/chickenpox/fact_sheet.htm',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/ya-no-se-recomienda-una-aspirina-diaria-para-muchos-adultos-mayores',
        'https://medlineplus.gov/spanish/druginfo/meds/a682399-es.html',
        'https://medlineplus.gov/spanish/ency/article/007612.htm'
    ],
    [
        'https://factual.afp.com/doc.afp.com.32G37YJ',
        'https://factual.afp.com/doc.afp.com.33E33FU',
        'https://medlineplus.gov/spanish/ency/article/001214.htm',
        'https://chequeado.com/el-explicador/es-falso-que-la-insulina-natural-de-3-hierbas-sirve-para-tratar-la-diabetes-tipo-2/',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/cambios-de-estilo-de-vida-son-clave-para-retrasar-o-prevenir-la-diabetes-tipo-2',
        'https://medlineplus.gov/spanish/diabetes.html'
    ],
    [
        'https://medlineplus.gov/spanish/druginfo/meds/a619019-es.html',
        'https://medlineplus.gov/spanish/druginfo/natural/843.html',
        'https://factual.afp.com/doc.afp.com.32FK4DA',
        'https://factual.afp.com/consumir-mucha-vitamina-c-no-impide-contagiarse-del-nuevo-coronavirus',
        'https://www.france24.com/es/20200620-cadena-falsa-aspirina-limon-miel-cura-covid19',
        'https://www.france24.com/es/20200325-factcheck-seg%C3%BAn-expertos-es-ineficaz-el-ant%C3%ADdoto-recomendado-por-maduro-contra-el-coronavirus',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/reflujo-de-acido-gastrico-acidez-estomacal-y-enfermedad-por-reflujo-gastroesofagico-erge-cual-es-la-diferencia',
    ],
    [
        'https://maldita.es/malditobulo/20201127/coronavirus-bulos-pandemia-prevenir-virus-covid-19/',
        'https://factual.afp.com/doc.afp.com.32H24NY',
        'https://factual.afp.com/marie-genries',
        'https://factual.afp.com/afp-belgique',
        'https://fakenewsvenezuela.org/nosotros/dicen-de-nosotros-2/',
        'https://laud.udistrital.edu.co/noticias/actualidad',
        'https://www.bbc.com/mundo/resources/idt-21c2c5c6-3973-405e-be7b-2c5ec95a4784',
        'https://www.eluniversal.com.co/salud/jugo-verde-es-bueno-tomarlo-todos-los-dias-esto-dicen-los-expertos-LX6918863'
    ],
    [
        'https://factual.afp.com/verificaciones-de-la-afp-en-espanol-sobre-el-nuevo-coronavirus',
        'https://factual.afp.com/las-falsas-afirmaciones-del-medico-canadiense-que-dijo-que-el-covid-19-es-una-farsa',
        'https://conecta.tec.mx/es/noticias/nacional/salud/mitos-y-preguntas-usuales-del-coronavirus-que-causa-el-covid-19',
        'https://medlineplus.gov/spanish/ency/patientinstructions/000452.htm',
        'https://medlineplus.gov/spanish/ency/patientinstructions/000447.htm',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/seis-maneras-de-comunicarse-mejor-al-usar-una-mascarilla',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/investigacion-inicial-prueba-la-reutilizacion-de-la-mascarilla-n95-para-el-covid-19'
    ],
    [
        'https://factual.afp.com/las-vacunas-de-arn-mensajero-en-estudio-para-prevenir-el-covid-19-no-modifican-el-adn-humano',
        'https://chequeado.com/el-explicador/son-falsas-varias-afirmaciones-del-medico-luis-martinez-que-circulan-en-videos-virales/',
        'https://factual.afp.com/doc.afp.com.33QD4X4',
        'https://www.cdc.gov/vaccines/parents/diseases/mumps-sp.html',
        'https://medlineplus.gov/spanish/ency/article/001526.htm',
        'https://medlineplus.gov/spanish/autismspectrumdisorder.html',
        'https://www.cdc.gov/spanish/index.html'
    ],
    [
        'https://medlineplus.gov/spanish/druginfo/meds/a682878-es.html',
        'https://factual.afp.com/los-ataques-cardiacos-y-derrames-no-se-evitan-al-tomar-agua-en-momentos-concretos-del-dia',
        'https://medlineplus.gov/spanish/bloodthinners.html',
        'https://magazine.medlineplus.gov/es/art%C3%ADculo/ya-no-se-recomienda-una-aspirina-diaria-para-muchos-adultos-mayores',
        'https://www.fda.gov/consumers/free-publications-women/salud-cardiaca-heart-health',
        'https://factual.afp.com/verificaciones-de-la-afp-en-espanol-sobre-el-nuevo-coronavirus',
        'https://www.france24.com/es/20200620-cadena-falsa-aspirina-limon-miel-cura-covid19'
    ]
]
noticias = []
for i in urls:
    noticias.append(obtener_noticias_completas(i))

In [23]:
def crear_bm25(algoritmo:str):
    impl = {'okapi': BM25_okapi, 'fast': BM25_fast, 'sparse': BM25_sparse, 'manual': BM25_manual}
    return impl[algoritmo]

In [55]:
querys = [
    'beber agua de limón cura el cáncer',
    'usar cubrebocas provoca neumonía',
    'comer cebolla cura el coronavirus',
    'el coronavirus se cura tomando aspirina',
    'la diabetes se cura con insulina',
    'el cáncer se cura tomando agua de limón',
    'la viruela del mono es provocada por la vacunación anticovid',
    'usar cubrebocas previene el covid-19',
    'las vacunas provocan autismo',
    'tomar aspirina previene ataques al corazón'
]
print('[')
for i in range(len(urls)):
    print('{\n\"query\":\"', querys[i] + '\",')
    for alg in ['manual', 'okapi', 'fast', 'sparse']:
        print('\"' + alg + '\": [')
        bm25 = crear_bm25(alg)()
        bm25.setQuery(query=querys[i])
        bm25.setDocs(noticias[i])
        noticias_ordenadas = bm25.apply()
        print('],\n')
    print('},')
print(']')

[
{
"query":" beber agua de limón cura el cáncer",
"manual": [
{"score": 4.370599999999999 ,
"título":" Beber agua caliente, sola o con limón, no cura el cáncer | Factual ",
"url":" https://factual.afp.com/beber-agua-caliente-sola-o-con-limon-no-cura-el-cancer "},
{"score": 4.369899999999999 ,
"título":" Consumir mucha vitamina C no impide contagiarse del nuevo coronavirus | Factual ",
"url":" https://factual.afp.com/consumir-mucha-vitamina-c-no-impide-contagiarse-del-nuevo-coronavirus "},
{"score": 4.3690999999999995 ,
"título":" Citrato de magnesio: MedlinePlus medicinas ",
"url":" https://medlineplus.gov/spanish/druginfo/meds/a619019-es.html "},
{"score": 4.366 ,
"título":" Reflujo de ácido gástrico, acidez estomacal y enfermedad por reflujo gastroesofágico (ERGE): ¿Cuál es la diferencia? | NIH MedlinePlus Magazine ",
"url":" https://magazine.medlineplus.gov/es/art%C3%ADculo/reflujo-de-acido-gastrico-acidez-estomacal-y-enfermedad-por-reflujo-gastroesofagico-erge-cual-es-la-diferenci

Building inverted index: 100%|██████████████████████████████████████████████████████████████████████████████| 1950/1950 [00:00<00:00, 7622.74it/s]


{"score": 4.8246493 ,
"título":" Cinco sugerencias para las personas con alopecia areata | NIH MedlinePlus Magazine ",
"url":" https://magazine.medlineplus.gov/es/art%C3%ADculo/cinco-sugerencias-para-las-personas-con-alopecia-areata "},
{"score": 4.778391 ,
"título":" Consumir mucha vitamina C no impide contagiarse del nuevo coronavirus | Factual ",
"url":" https://factual.afp.com/consumir-mucha-vitamina-c-no-impide-contagiarse-del-nuevo-coronavirus "},
{"score": 4.336999 ,
"título":" AFP Noticias | Facebook ",
"url":" https://www.facebook.com/AFPespanol/posts/d41d8cd9/2517545981862495/ "},
{"score": 0.86439365 ,
"título":" Beber agua caliente, sola o con limón, no cura el cáncer | Factual ",
"url":" https://factual.afp.com/beber-agua-caliente-sola-o-con-limon-no-cura-el-cancer "},
{"score": 0.79741675 ,
"título":" Citrato de magnesio: MedlinePlus medicinas ",
"url":" https://medlineplus.gov/spanish/druginfo/meds/a619019-es.html "},
{"score": 0.6832892 ,
"título":" Cola de Caballo: Med

Building inverted index: 100%|██████████████████████████████████████████████████████████████████████████████| 6389/6389 [00:00<00:00, 8398.54it/s]


{"score": 4.2035055 ,
"título":" Uso de guantes en el hospital: MedlinePlus enciclopedia médica ",
"url":" https://medlineplus.gov/spanish/ency/patientinstructions/000452.htm "},
{"score": 4.001867 ,
"título":" COVID-19, una emergencia de salud pública mundial - PMC ",
"url":" https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102523/ "},
{"score": 3.1457267 ,
"título":" Neumonía en adultos, adquirida en la comunidad: MedlinePlus enciclopedia médica ",
"url":" https://medlineplus.gov/spanish/ency/article/000145.htm "},
{"score": 2.5094197 ,
"título":" Verificaciones de la AFP en español sobre el nuevo coronavirus | Factual ",
"url":" https://factual.afp.com/verificaciones-de-la-afp-en-espanol-sobre-el-nuevo-coronavirus "},
{"score": 1.4268312 ,
"título":" Las mascarillas limpias y bien manipuladas no provocan neumonía | Factual ",
"url":" https://factual.afp.com/las-mascarillas-limpias-y-bien-manipuladas-no-provocan-neumonia "},
{"score": 1.0302745 ,
"título":" Facebook ",
"url":" https://w

...

{"score": 4.723771 ,
"título":" Uso de guantes en el hospital: MedlinePlus enciclopedia médica ",
"url":" https://medlineplus.gov/spanish/ency/patientinstructions/000452.htm "},
{"score": 3.7620208 ,
"título":" Verificaciones de la AFP en español sobre el nuevo coronavirus | Factual ",
"url":" https://factual.afp.com/verificaciones-de-la-afp-en-espanol-sobre-el-nuevo-coronavirus "},
{"score": 2.6790824 ,
"título":" Mitos y preguntas usuales del coronavirus que causa el COVID-19 | Tecnológico de Monterrey ",
"url":" https://conecta.tec.mx/es/noticias/nacional/salud/mitos-y-preguntas-usuales-del-coronavirus-que-causa-el-covid-19 "},
{"score": 0.98122996 ,
"título":" Investigación inicial prueba la reutilización de la mascarilla N95 para el COVID-19 | NIH MedlinePlus Magazine ",
"url":" https://magazine.medlineplus.gov/es/art%C3%ADculo/investigacion-inicial-prueba-la-reutilizacion-de-la-mascarilla-n95-para-el-covid-19 "},
{"score": 0.9656128 ,
"título":" Equipo de protección personal: Med

Building inverted index: 100%|██████████████████████████████████████████████████████████████████████████████| 1996/1996 [00:00<00:00, 6335.16it/s]


{"score": 1.7213671 ,
"título":" Trastorno del espectro autista: MedlinePlus en español ",
"url":" https://medlineplus.gov/spanish/autismspectrumdisorder.html "},
{"score": 1.6314915 ,
"título":" Las paperas y la vacuna (inyectable) | CDC ",
"url":" https://www.cdc.gov/vaccines/parents/diseases/mumps-sp.html "},
{"score": 0.92624694 ,
"título":" Centros para el Control y la Prevención de Enfermedades CDC ",
"url":" https://www.cdc.gov/spanish/index.html "},
{"score": 0.85242516 ,
"título":" Trastorno del espectro autista: MedlinePlus enciclopedia médica ",
"url":" https://medlineplus.gov/spanish/ency/article/001526.htm "},
{"score": 0.8080822 ,
"título":" Casos de autismo han aumentado históricamente, pero no tiene que ver con la vacunación | Factual ",
"url":" https://factual.afp.com/doc.afp.com.33QD4X4 "},
{"score": 0.6461586 ,
"título":" Las vacunas de ARN mensajero en estudio para prevenir el covid-19 no modifican el ADN humano | Factual ",
"url":" https://factual.afp.com/las-vacun

Building inverted index: 100%|██████████████████████████████████████████████████████████████████████████████| 5831/5831 [00:00<00:00, 9197.03it/s]


{"score": 2.65399 ,
"título":" Salud cardíaca [Heart Health] | FDA ",
"url":" https://www.fda.gov/consumers/free-publications-women/salud-cardiaca-heart-health "},
{"score": 1.6444932 ,
"título":" Los ataques cardíacos y derrames no se evitan al tomar agua en momentos concretos del día | Factual ",
"url":" https://factual.afp.com/los-ataques-cardiacos-y-derrames-no-se-evitan-al-tomar-agua-en-momentos-concretos-del-dia "},
{"score": 1.144625 ,
"título":" Verificaciones de la AFP en español sobre el nuevo coronavirus | Factual ",
"url":" https://factual.afp.com/verificaciones-de-la-afp-en-espanol-sobre-el-nuevo-coronavirus "},
{"score": 1.1338097 ,
"título":" Cadenas falsas sobre uso de Aspirina, limón y miel para curar el Covid-19 generan acaparamiento ",
"url":" https://www.france24.com/es/20200620-cadena-falsa-aspirina-limon-miel-cura-covid19 "},
{"score": 1.1328605 ,
"título":" Ya no se recomienda una aspirina diaria para muchos adultos mayores | NIH MedlinePlus Magazine ",
"url":" h